In [1]:
import collections
import functools
import itertools
import os
from pathlib import Path
import random
import re
import string
import sys
from timeit import default_timer
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from wordcloud import WordCloud

sys.path.append('../')
warnings.filterwarnings('ignore')

In [14]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/skp340/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import sm_analysis
import sm_analysis.topic_modeling as topic

In [5]:
from sm_analysis import youtube
from sm_analysis.youtube import api
from sm_analysis.youtube import channels
from sm_analysis.youtube import videos
from sm_analysis.youtube import utils

# Retrieve comments

In [59]:
search = api.Search("jacobin magazine")

In [60]:
for page in range(5):
    result = search.api_request()
    search.parse(result)

In [61]:
df = search.to_dataframe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   kind                  25 non-null     object
 1   channelId             25 non-null     object
 2   publishedAt           25 non-null     object
 3   title                 25 non-null     object
 4   description           25 non-null     object
 5   thumbnails            25 non-null     object
 6   channelTitle          25 non-null     object
 7   liveBroadcastContent  25 non-null     object
 8   publishTime           25 non-null     object
 9   videoId               23 non-null     object
dtypes: object(10)
memory usage: 2.1+ KB


In [70]:
videos = df.where(df["channelTitle"]=="Jacobin").dropna()

In [78]:
vids = videos["videoId"]

In [84]:
threads = [api.CommentThreads(vids.iloc[i]) for i in range(len(vids))]

In [85]:
results = [thread.api_request() for thread in threads]

In [88]:
for thread, result in zip(threads, results):
    thread.parse(result)

In [91]:
df = threads[0].to_dataframe()
for i in range(1,len(threads)):
    df = df.append(threads[i].to_dataframe())

In [92]:
df

,videoId,textDisplay,textOriginal,authorDisplayName,authorProfileImageUrl,authorChannelUrl,canRate,viewerRating,likeCount,publishedAt,updatedAt,canReply,totalReplyCount,isPublic,authorchannelId,parentId
id,,,,,,,,,,,,,,,,
UgzzuQVi66aJA5EQ6M14AaABAg,XnvpdCPeIJU,I agree with just about everything that Mister...,I agree with just about everything that Mister...,Michael Kilcommons,https://yt3.ggpht.com/ytc/AMLnZu-bFBP7EjPScUef...,http://www.youtube.com/channel/UCgiPbAaHBac_ie...,True,none,0,2022-09-05T16:01:29Z,2022-09-05T16:01:29Z,True,0,True,UCgiPbAaHBac_ierA1rpqwUg,UgzzuQVi66aJA5EQ6M14AaABAg
UgwXnDFD9tx_QzcFpqB4AaABAg,XnvpdCPeIJU,Great job Jen with this interview of Dr. Gerst...,Great job Jen with this interview of Dr. Gerst...,AM,https://yt3.ggpht.com/7SAdcerpBlNMSwVr6LKBTIpZ...,http://www.youtube.com/channel/UC0qS1_HKZJADb5...,True,none,0,2022-08-30T15:58:33Z,2022-08-30T15:58:33Z,True,0,True,UC0qS1_HKZJADb5Nn9k-KjlA,UgwXnDFD9tx_QzcFpqB4AaABAg
Ugwl976lg7VgU4DSxSd4AaABAg,XnvpdCPeIJU,Enjoyed the interview!,Enjoyed the interview!,Russell Gallman,https://yt3.ggpht.com/ytc/AMLnZu8Rv8jXuauyYFmQ...,http://www.youtube.com/channel/UCd5XaRJREEuLtf...,True,none,0,2022-08-28T23:38:38Z,2022-08-28T23:38:38Z,True,0,True,UCd5XaRJREEuLtfYXPaEC86Q,Ugwl976lg7VgU4DSxSd4AaABAg
Ugz_pf9i7YuC5tJ8p2l4AaABAg,XnvpdCPeIJU,Get lobbyists and insider trading out of polit...,Get lobbyists and insider trading out of polit...,Adam Brock,https://yt3.ggpht.com/ytc/AMLnZu8tr_LGxqWo8iBx...,http://www.youtube.com/channel/UCMXbC79tRzitOs...,True,none,0,2022-08-23T14:12:56Z,2022-08-23T14:13:53Z,True,0,True,UCMXbC79tRzitOsSmRhnJfeg,Ugz_pf9i7YuC5tJ8p2l4AaABAg
Ugxel72Il2EK95bTz4t4AaABAg,XnvpdCPeIJU,Fck Neoliberalism!!!!,Fck Neoliberalism!!!!,Noah Woodhope,https://yt3.ggpht.com/ytc/AMLnZu9nKKilcG7Gdpf4...,http://www.youtube.com/channel/UCgv6f2oe77M5D3...,True,none,0,2022-08-07T18:40:19Z,2022-08-07T18:40:19Z,True,0,True,UCgv6f2oe77M5D3Nwp2Wje7A,Ugxel72Il2EK95bTz4t4AaABAg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UgymP1A7hjz0cqf6pKt4AaABAg,b-zkEAq-5Dc,I think the fact that non-profit employees ten...,I think the fact that non-profit employees ten...,David Breitowitz,https://yt3.ggpht.com/PwfvLWbnLOHasa3vUmNJMQao...,http://www.youtube.com/channel/UCytAdny2KqKX5m...,True,none,3,2022-06-29T03:08:01Z,2022-06-29T03:08:01Z,True,3,True,UCytAdny2KqKX5m7xdKtsN4w,UgymP1A7hjz0cqf6pKt4AaABAg
UgxweMzJCcEULvJYuZp4AaABAg,b-zkEAq-5Dc,Pyramidic power structure. It will be the dea...,Pyramidic power structure. It will be the dea...,bpalpha,https://yt3.ggpht.com/ytc/AMLnZu-HPjgG29jIpMXS...,http://www.youtube.com/channel/UCfE_G0x3iOJheH...,True,none,1,2022-06-28T22:48:45Z,2022-06-28T23:01:44Z,True,0,True,UCfE_G0x3iOJheHZV_WtdzDg,UgxweMzJCcEULvJYuZp4AaABAg
UgzVl-koIse-ARWoOfd4AaABAg,b-zkEAq-5Dc,The article was painfully spot on. I now emai...,The article was painfully spot on. I now emai...,Shani Handel,https://yt3.ggpht.com/ytc/AMLnZu_Q2ZhBAPbtNALX...,http://www.youtube.com/channel/UCAqGDSiRqfDDMr...,True,none,14,2022-06-28T22:24:57Z,2022-06-28T22:24:57Z,True,1,True,UCAqGDSiRqfDDMr4BX7sUkTw,UgzVl-koIse-ARWoOfd4AaABAg


# Topic modeling

In [93]:
string_agg = lambda x: list(x)

comment_text = (df
               .groupby('authorDisplayName')[['textOriginal']]
               .agg(string_agg)
              )

In [94]:
comment_text

,textOriginal
authorDisplayName,
A DEK,[Will he mention US oligarchs?]
A. Lerner,"[The US was built upon slavery, they have simp..."
AM,[Great job Jen with this interview of Dr. Gers...
Aaron Wellman,[i just find all of the 'woke' / heavily ident...
Abject Individual,[58:41]
...,...
ultraviolet iris,[Yep this is why MLK was killed]
umperthay,"[NATO is just another tool of the globalists, ..."
wv watson,[Lol @ uvalde blue lives matter]


In [95]:
user_tokens = (comment_text['textOriginal']
               .map(topic.make_tokens)
               .map(topic.long_list))
user_tokens.head(3)

authorDisplayName
A DEK                                     [mention, oligarchs]
A. Lerner    [built, slavery, simply, refined, system, incl...
AM           [great, job, jen, interview, dr, gerstle, read...
Name: textOriginal, dtype: object

# Topic extraction

In [102]:
stop_words = [*stopwords.words(),
              'jen', 'gerstle','gary','jacobin',
             ]
tokenize_partial = functools.partial(topic.tokenize_string, 
                                     stop_words=stop_words)
preprocess_partial = functools.partial(topic.preprocess_string, 
                                       special_tokens=False)

In [103]:
vectorizer = CountVectorizer(analyzer='word',
                             strip_accents='ascii',
                             stop_words=stop_words,
                             ngram_range=(1,2),
                             preprocessor=preprocess_partial,
                             tokenizer=tokenize_partial,
                            )

documents_list = (comment_text['textOriginal']
                    .map(topic.long_string)
                   )

In [104]:
print(documents_list[:5])

authorDisplayName
A DEK                                    Will he mention US oligarchs?
A. Lerner            The US was built upon slavery, they have simpl...
AM                   Great job Jen with this interview of Dr. Gerst...
Aaron Wellman        i just find all of the 'woke' / heavily identi...
Abject Individual                                                58:41
Name: textOriginal, dtype: object


In [105]:
# tokenize + construct the frequency matrix from tokens
matrix = vectorizer.fit_transform(documents_list)

In [106]:
# get token names + frequencies
ngrams = vectorizer.get_feature_names()
ngram_frequencies = matrix.sum(axis=0).tolist()[0]
freq_dict = dict(zip(ngrams, ngram_frequencies))

In [107]:
print(sorted(freq_dict.items(), key=lambda item: item[1], reverse=True)[:20])

[('class', 40), ('pay', 38), ('left', 32), ('power', 27), ('make', 26), ('system', 26), ('bernie', 23), ('issues', 22), ('working', 22), ('world', 21), ('democratic', 19), ('great', 19), ('money', 19), ('time', 19), ('back', 18), ('economic', 18), ('america', 17), ('problem', 17), ('social', 17), ('working class', 17)]


# Text classification

In [ ]:
def preds_from_list(strings_list, classifier):
    preds = [classifier.from_text(string) for string in strings_list]
    pairs = list(zip(strings_list, preds))

    return sorted(pairs, key=lambda p: p[1])

def pred_from_text(string, classifier):
    return classifier.from_text(string)

## Basic sentiment classifier (Hugging Face transformers)

In [108]:
import torch
import transformers
from transformers import pipeline

2022-09-14 05:23:03.056988: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
classifier = pipeline("sentiment-analysis")

sentence = df.iloc[0]["originalText"]
print(sentence)
result = classifier(sentence)
print(result)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


## Microsoft Toxigen

Microsoft model pre-trained on Toxigen hate speech dataset

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from toxigen import alice
from toxigen.pretrained_classifiers import ToxDectRoBERTa, HateBERT

# Load a pretrained hate speech classifier
tox_classifier = ToxDectRoBERTa()

prob = tox_classifier.from_text(sentence)
print("input text: {}\n"
      "Probability the input text was toxic according to ToxDectRoBERTa: {:3f}".format(sentence, prob))